In [2]:
from pandas.io import gbq
import pandas as pd
import matplotlib.pyplot as plt

bq_project = "thesis-jaeyoon"

In [26]:
gp_query="""

select
	p.publication_number,
	p.application_number,
	p.country_code,
  p.publication_date,
  CAST(FLOOR(p.filing_date / 10000) AS INT64) AS filing_year,
  CAST(FLOOR(p.priority_date / 10000) AS INT64) AS priority_year,
  p.family_id,
  p.entity_status,
	STRING_AGG(DISTINCT ipcs.code) as ipc_codes,
  STRING_AGG(DISTINCT uspcs.code) as uspc_codes,
  STRING_AGG(DISTINCT pc.application_number) as priority_application_number,
  STRING_AGG(DISTINCT cit.publication_number) as citations,
  STRING_AGG(DISTINCT ah.name, "||") as assignees_harmonized,
  STRING_AGG(DISTINCT title.text, "||") as title_text,
  STRING_AGG(DISTINCT abstract.text, "||") as abstract_text
FROM
  `patents-public-data.patents.publications` AS p
LEFT JOIN UNNEST(p.ipc) AS ipcs
LEFT JOIN UNNESt(p.uspc) AS uspcs
LEFT JOIN UNNEST(p.assignee) AS assignees
LEFT JOIN UNNEST(p.title_localized) AS title
LEFT JOIN UNNEST(p.abstract_localized) AS abstract
LEFT JOIN UNNEST(p.priority_claim) AS pc 
LEFT JOIN UNNEST(p.citation) as cit
LEFT JOIN UNNEST(p.assignee_harmonized) as ah
LEFT JOIN UNNEST(p.description_localized) as description
where 
  pc.application_number = p.application_number AND 
  (
  ipcs.code LIKE '%A61J1/%' OR ipcs.code LIKE '%A61J3/%' OR
  ipcs.code LIKE '%A61K9/%' OR ipcs.code LIKE '%A61K31/%' OR
  ipcs.code LIKE '%A61K33/%' OR ipcs.code LIKE '%A61K35/%' OR
  ipcs.code LIKE '%A61K36/%' OR ipcs.code LIKE '%A61K41/%' OR
  ipcs.code LIKE '%A61K49/%' OR ipcs.code LIKE '%A61K51/%' OR
  ipcs.code LIKE '%A61K39/%' OR ipcs.code LIKE '%A61K45/%' OR
  ipcs.code LIKE '%A61K47/%' OR ipcs.code LIKE '%A61K48/%' OR
  ipcs.code LIKE '%A61P/%' OR
  uspcs.code LIKE '%424/%' OR uspcs.code LIKE '%514/%' 
  )
GROUP BY 
  p.publication_number, p.application_number, p.country_code, p.publication_date, p.filing_date,
	p.priority_date, p.family_id, p.entity_status
"""

In [27]:
gp_df = gbq.read_gbq(
            query=gp_query,
            project_id=bq_project,
            verbose=True,
            dialect='standard')

/Users/jaeyoonjung/anaconda3/lib/python3.6/site-packages/pandas/io/gbq.py:149: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  credentials=credentials, verbose=verbose, private_key=private_key)


In [28]:
gp_df.to_csv("data/priority_claims.csv", sep="\t", index=False)

In [30]:
ob_patent_dir = "orangebook_chembl.csv"
ob_patent = pd.read_csv(ob_patent_dir, sep="\t")

In [31]:
where_clause = ",".join("'" + ob_patent.publication_number.unique() + "'")

In [32]:
ob_query = '''
select
	p.publication_number,
	p.application_number,
	p.country_code,
  p.publication_date,
  CAST(FLOOR(p.filing_date / 10000) AS INT64) AS filing_year,
  CAST(FLOOR(p.priority_date / 10000) AS INT64) AS priority_year,
  p.family_id,
  p.entity_status,
	STRING_AGG(DISTINCT ipcs.code) as ipc_codes,
  STRING_AGG(DISTINCT uspcs.code) as uspc_codes,
  STRING_AGG(DISTINCT pc.application_number) as priority_application_number,
  STRING_AGG(DISTINCT cit.publication_number) as citations,
  STRING_AGG(DISTINCT ah.name, "||") as assignees_harmonized,
  STRING_AGG(DISTINCT title.text, "||") as title_text,
  STRING_AGG(DISTINCT abstract.text, "||") as abstract_text
FROM
  `patents-public-data.patents.publications` AS p
LEFT JOIN UNNEST(p.ipc) AS ipcs
LEFT JOIN UNNESt(p.uspc) AS uspcs
LEFT JOIN UNNEST(p.assignee) AS assignees
LEFT JOIN UNNEST(p.title_localized) AS title
LEFT JOIN UNNEST(p.abstract_localized) AS abstract
LEFT JOIN UNNEST(p.priority_claim) AS pc 
LEFT JOIN UNNEST(p.citation) as cit
LEFT JOIN UNNEST(p.assignee_harmonized) as ah
LEFT JOIN UNNEST(p.description_localized) as description
WHERE
    p.publication_number IN
    (
    {}
    )
GROUP BY 
p.publication_number, p.application_number, p.country_code, p.publication_date, p.filing_date,
	p.priority_date, p.family_id, p.entity_status
'''.format(where_clause)

In [33]:
ob_df = gbq.read_gbq(
            query=ob_query,
            project_id=bq_project,
            verbose=True,
            dialect='standard')

/Users/jaeyoonjung/anaconda3/lib/python3.6/site-packages/pandas/io/gbq.py:149: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  credentials=credentials, verbose=verbose, private_key=private_key)


In [34]:
ob_df = pd.merge(left = ob_df, right = ob_patent[["publication_number", "mesh_heading"]], on ="publication_number")

In [35]:
ob_df.to_csv("data/ob_patents.csv", sep='\t', index=False)